In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from tqdm.auto import tqdm

from modules.data_manage.load_mnist import load_mnist
from modules.grammar.count_accuracy import CountAccuracy
from modules.grammar.grammar import Grammar
from modules.grammar.merge.stupid_merge_same_nodes import stupid_merge_same_nodes
from modules.grammar.node import merge_nodes
from modules.plots.draw_imgs import draw_imgs
from modules.plots.plot_random_from_grammar import plot_random_from_grammar
from modules.utils import inject_jupyter_style

inject_jupyter_style()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
x_train, y_train, x_test, y_test = load_mnist(size=1000)
# draw_imgs(x_train, (2, 7), random=False)

Train: (1000, 32, 32) (1000,)
Test:  (1000, 32, 32) (1000,)


In [85]:
grammar = Grammar(colors=x_train.max() + 1)
metric = CountAccuracy(
    grammar=grammar,
    train=x_train[y_train == 0],
    valid=x_test[y_test == 0],
    wrong_train=[x_train[y_train == i] for i in range(1, 10)],
    wrong_valid=[x_test[y_test == i] for i in range(1, 10)],
)
grammar.add_images(x_train[y_train == 0])

  0%|          | 0/97 [00:00<?, ?it/s]

In [86]:
stupid_merge_same_nodes(grammar)

(19780, 3638)

In [21]:
nodes = grammar.set_height_for_all_nodes()

height           0   1   2    3    4    5    6    7    8    9   10  11
number of nodes   2   4  30  182  431  902  839  589  369  194  97   1


In [22]:
metric.count_metrics()

{'train': 0.7731958762886598,
 'valid': 0.0,
 'wrong train': 0.0,
 'wrong valid': 0.0,
 'wrong train 1': 0.0,
 'wrong train 2': 0.0,
 'wrong train 3': 0.0,
 'wrong train 4': 0.0,
 'wrong train 5': 0.0,
 'wrong train 6': 0.0,
 'wrong train 7': 0.0,
 'wrong train 8': 0.0,
 'wrong train 9': 0.0,
 'wrong valid 1': 0.0,
 'wrong valid 2': 0.0,
 'wrong valid 3': 0.0,
 'wrong valid 4': 0.0,
 'wrong valid 5': 0.0,
 'wrong valid 6': 0.0,
 'wrong valid 7': 0.0,
 'wrong valid 8': 0.0,
 'wrong valid 9': 0.0}

In [11]:
merge_nodes(nodes[1][0], nodes[1][1])
merge_nodes(nodes[1][2], nodes[1][3])

stupid_merge_same_nodes(grammar)

nodes = grammar.set_height_for_all_nodes()
merge_nodes(nodes[1][0], nodes[1][1])

height           0   1   2    3    4    5    6    7    8    9   10  11
number of nodes   2   2  15  145  402  886  838  589  369  194  97   1


In [14]:
stupid_merge_same_nodes(grammar)
nodes = grammar.set_height_for_all_nodes()

height           0   1   2    3    4    5    6    7    8    9   10  11
number of nodes   2   1   9  131  393  881  838  589  369  194  97   1


In [17]:
metric.count_metrics()

{'train': 0.061855670103092786,
 'valid': 0.0,
 'wrong train': 0.0,
 'wrong valid': 0.0,
 'wrong train 1': 0.0,
 'wrong train 2': 0.0,
 'wrong train 3': 0.0,
 'wrong train 4': 0.0,
 'wrong train 5': 0.0,
 'wrong train 6': 0.0,
 'wrong train 7': 0.0,
 'wrong train 8': 0.0,
 'wrong train 9': 0.0,
 'wrong valid 1': 0.0,
 'wrong valid 2': 0.0,
 'wrong valid 3': 0.0,
 'wrong valid 4': 0.0,
 'wrong valid 5': 0.0,
 'wrong valid 6': 0.0,
 'wrong valid 7': 0.0,
 'wrong valid 8': 0.0,
 'wrong valid 9': 0.0}